# Web Research Sub-Agent Test Notebook

This notebook tests the web research sub-agent, which is responsible for:
- **Finding Information**: Searching for relevant, reliable sources
- **Gathering Data**: Collecting facts, statistics, and quotes
- **Documenting Sources**: Keeping detailed records of where information came from
- **Assessing Initial Quality**: Noting if sources seem reliable or questionable

The main agent uses this sub-agent to:
- Research why stock prices are moving
- Analyze recent price action, volume, and volatility
- Investigate company-specific news, earnings, and guidance
- Gather sector trends, competitor info, and macro factors
- Collect datasets for the analysis agent to process

## Prerequisites

This notebook connects to the LangGraph server, which provides:
- **Cloud-hosted store** - Persistent memory across sessions (same as LangSmith Studio)
- **Cloud-hosted checkpointer** - Conversation state persistence

### Setup Steps

1. **Start the LangGraph server** (from the `deep-agent/` directory):
   ```bash
   cd deep-agent
   langgraph dev
   ```

2. **Wait for the server** to be ready at `http://localhost:2024`

3. **Run this notebook** - it connects to the same agent as LangSmith Studio

### Why This Architecture?

When you run `langgraph dev`, LangGraph connects to LangSmith's cloud infrastructure.
This means:
- Agent state persists across sessions
- No local PostgreSQL database required
- Same agent instance as LangSmith Studio

## Setup

In [ ]:
# Set to the assistant_id to use for this notebook
MANUAL_ASSISTANT_ID = "3e46236d-ede5-5fd4-91d9-6fd4977ad898"

In [1]:
# Ensure scratchpad folders exist and are empty
from pathlib import Path
import shutil

scratchpad = Path("../scratchpad")
for folder in ["data", "images", "notes", "plots", "reports"]:
    path = scratchpad / folder
    if path.exists():
        shutil.rmtree(path)
    path.mkdir(parents=True)
    
print("Scratchpad folders ready (data, images, notes, plots, reports)")

Scratchpad folders ready (data, images, notes, plots, reports)


In [2]:
from langgraph_sdk import get_sync_client
from dotenv import load_dotenv
import os

load_dotenv()

# Connect to LangGraph server (must have `langgraph dev` running)
LANGGRAPH_URL = "http://localhost:2024"
ASSISTANT_ID = MANUAL_ASSISTANT_ID

if not ASSISTANT_ID:
    raise SystemExit("Set MANUAL_ASSISTANT_ID before running this notebook.")

try:
    client = get_sync_client(url=LANGGRAPH_URL)
    assistants = client.assistants.search(limit=20)
    selected = next((a for a in assistants if a.get("assistant_id") == ASSISTANT_ID), None)
    if not selected:
        raise RuntimeError(f"Assistant {ASSISTANT_ID} not found. Ensure it exists on the LangGraph server.")

    print(f"Connected to LangGraph server at {LANGGRAPH_URL}")
    print(f"Using assistant_id: {ASSISTANT_ID}")
except Exception as e:
    print(f"Could not connect to LangGraph server at {LANGGRAPH_URL}")
    print(f"Error: {e}")
    print("Make sure to run 'langgraph dev' from the deep-agent/ directory first.")
    raise SystemExit(1)


Connected to LangGraph server at http://localhost:2024
Available assistants:
  - 91e1bc6c-3ce8-58e1-9506-d229476e35f8: credibility-agent
  - 3e46236d-ede5-5fd4-91d9-6fd4977ad898: web-research-agent (selected)
  - fe096781-5601-53d2-b2f6-0d3403f7e9ca: agent
  - 854aa6d0-4ddb-5980-837f-8db6045be50f: analysis-agent
Using assistant_id: 3e46236d-ede5-5fd4-91d9-6fd4977ad898


## Helper Function to Test the Agent

In [3]:
import time
from IPython.display import display, Markdown


def truncate(text, limit=2000):
    return text[:limit] + "\n..." if len(text) > limit else text


def test_web_research_agent(message: str):
    """Run the web research agent via LangGraph SDK and display all intermediate steps."""
    # Always start a fresh thread for a clean run
    thread = client.threads.create()
    thread_id = thread.get("thread_id") or f"web-{int(time.time())}"

    display(Markdown(f"## 📝 Task\n```\n{message.strip()}\n```\n---"))

    final_response = None

    for chunk in client.runs.stream(
        thread_id=thread_id,
        assistant_id=ASSISTANT_ID,
        input={"messages": [{"role": "user", "content": message}]},
        stream_mode="updates",
    ):
        if chunk.event == "updates":
            for node_name, node_output in chunk.data.items():
                messages = node_output.get("messages", []) if isinstance(node_output, dict) else []
                for msg in messages:
                    if not isinstance(msg, dict):
                        continue
                    msg_type = msg.get("type")

                    if msg_type == "ai" and msg.get("tool_calls"):
                        for tc in msg.get("tool_calls", []):
                            name = tc.get("name")
                            args = tc.get("args", {})
                            display(Markdown(f"### 🔧 Tool Call: `{name}`\n```json\n{truncate(str(args), 500)}\n```"))

                    elif msg_type == "tool":
                        content = msg.get("content", "")
                        display(Markdown(f"### 📤 Tool Response\n```\n{truncate(content)}\n```\n---"))

                    elif msg_type == "ai" and msg.get("content") and not msg.get("tool_calls"):
                        final_response = msg["content"]
                        display(Markdown(f"## ✅ Response\n{final_response}"))

    return final_response


---
# Example 1: Single Stock Research (Simple)

**Context**: A trader wants to understand why a specific stock moved significantly.

**Sub-agent role**: Research recent news and events related to the stock and provide cited findings.

In [4]:
# Example 1: Single stock research
example_1_message = """Research why Microsoft (MSFT) stock has been moving recently.

Focus on:
- Recent news or announcements
- Any earnings or guidance updates
- AI-related developments

Provide your findings with proper source citations.
"""

import time
response_1 = test_web_research_agent(example_1_message)


## 📝 Task
```
Research why Microsoft (MSFT) stock has been moving recently.

Focus on:
- Recent news or announcements
- Any earnings or guidance updates
- AI-related developments

Provide your findings with proper source citations.
```
---

### 🔧 Tool Call: `ls`
```json
{'path': '/memories'}
```

### 🔧 Tool Call: `web_search`
```json
{'query': 'Microsoft MSFT stock news December 2025', 'max_results': 8, 'topic': 'finance'}
```

### 🔧 Tool Call: `web_search`
```json
{'query': 'MSFT stock movement recent days December 2025', 'max_results': 8, 'topic': 'finance'}
```

### 🔧 Tool Call: `web_search`
```json
{'query': 'Microsoft earnings 2025 Q1 Q2 Q3 FY25 guidance AI', 'max_results': 8, 'topic': 'finance'}
```

...

### 📤 Tool Response
```
{"query": "MSFT stock movement recent days December 2025", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Financial Analysis for MSFT", "url": "https://finance.yahoo.com/quote/MSFT/", "content": "Stock: MSFT\nFinancial Analysis:\nLatest Open Price: 484.88 Latest Close Price: 486.57 Highest Close Price: 541.06 Lowest Close Price: 352.67 Average Close Price (2 years): 438.13 Standard Deviation of Close Price: 45.09 Volume Traded (2 years): 10733204361 Total Return (2 years): 31.80% Annualized Return (2 years): 14.81% CAGR (2 years): 14.81% Sharpe Ratio (2 years): 0.33 Max Drawdown (2 years): 0.03% ", "score": 0.98825, "raw_content": null}, {"url": "https://finance.yahoo.com/quote/MSFT/history/", "title": "Microsoft Corporation (MSFT) Stock Historical Prices & Data", "content": "Microsoft Corporation (MSFT) ; Dec 23, 2025, 484.88, 487.83 ; Dec 22, 2025, 486.12, 488.73 ; Dec 19, 2025, 487.36, 487.85 ; Dec 18, 2025, 478.19, 489.60 ...Read more", "score": 0.90791017, "raw_content": null}, {"url": "https://finance.yahoo.com/quote/MSFT/analysis/", "title": "Microsoft Corporation (MSFT) Analyst Ratings, Estimates & ...", "content": "Current Qtr. (Dec 2025), Next Qtr. (Mar 2026), Current Year (2026), Next Year (2027). No. of Analysts, 33, 33, 42, 43. Avg. Estimate, 3.86, 3.92, 15.75, 18.64.Read more", "score": 0.784336, "raw_content": null}, {"url": "https://finance.yahoo.com/quote/MSFT/key-statistics/", "title": "Microsoft Corporation (MSFT) Valuation Measures & ...", "content": "Stock Price History. Beta (5Y Monthly), 1.07. 52 Week Change 3, 10.60%. S&P 500 ... Dec 22, 2025, 7:00 PM EST. Prior: Prior: 2.8. New: New: -. PCE Price Index ...Read more", "score": 0.7738498, "raw_content": null}, {"url": "https://finance.yahoo.com/news/stock-market-news-dec-23-103000908.html", "title": "Stock Market News for Dec 23, 2025", "content": "U.S. stock markets closed higher on Monday to start the Christmas week. Wall Street generally remains positive i
...
```
---

### 📤 Tool Response
```
{"query": "Microsoft MSFT stock news December 2025", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Financial Analysis for MSFT", "url": "https://finance.yahoo.com/quote/MSFT/", "content": "Stock: MSFT\nFinancial Analysis:\nLatest Open Price: 484.88 Latest Close Price: 486.59 Highest Close Price: 541.06 Lowest Close Price: 352.67 Average Close Price (2 years): 438.13 Standard Deviation of Close Price: 45.09 Volume Traded (2 years): 10733204295 Total Return (2 years): 31.81% Annualized Return (2 years): 14.81% CAGR (2 years): 14.81% Sharpe Ratio (2 years): 0.33 Max Drawdown (2 years): 0.03% ", "score": 0.9878, "raw_content": null}, {"url": "https://finance.yahoo.com/news/microsoft-corporation-msft-azure-revenue-121646786.html", "title": "Microsoft Corporation's (MSFT) Azure Revenue Grew 39% ...", "content": "In its third-quarter 2025 investor letter, Baron Fifth Avenue Growth Fund highlighted stocks such as Microsoft Corporation (NASDAQ:MSFT). The one-month return of Microsoft Corporation (NASDAQ:MSFT) was -3.38%, and its shares gained 12.02% of their value over the last 52 weeks. On December 02, 2025, Microsoft Corporation (NASDAQ:MSFT) stock closed at $490.00 per share, with a market capitalization of $3.642 trillion. Baron Fifth Avenue Growth Fund stated the following regarding Microsoft Corporation (NASDAQ:MSFT) in its third quarter 2025 investor letter:. Microsoft Corporation (NASDAQ:MSFT) is in the second position on our list of 30 Most Popular Stocks Among Hedge Funds. As per our database, 312 hedge fund portfolios held Microsoft Corporation (NASDAQ:MSFT) at the end of the third quarter compared to 294 in the previous quarter. In the first quarter of fiscal year 2026, Microsoft Corporation (NASDAQ:MSFT) reported revenue of $77.7 billion, representing an 18% and 17% when adjusted for constant currency. In another article, we covered Microsoft Corporation (NASDAQ:MSFT) and shared the list of best dividend stocks hedge funds are
...
```
---

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/website_quality.txt', 'offset': 0, 'limit': 200}
```

### 📤 Tool Response
```
Error: File '/website_quality.txt' not found
```
---

## ✅ Response
1. **Summary of what I found (with citations)**

- Microsoft (MSFT) has traded recently around the high‑$480s, after pulling back from an all‑time high above $540 earlier this year, with a 2‑year total return of ~32% and 52‑week gain of about 11%. [Yahoo Finance – Quote](https://finance.yahoo.com/quote/MSFT/)  
- Short‑term moves in December 2025 appear mostly tied to broader market strength into year‑end and ongoing enthusiasm about Microsoft’s AI and cloud growth, rather than a single company‑specific shock. [Yahoo Finance – Quote](https://finance.yahoo.com/quote/MSFT/) [Yahoo Finance – Stock Market News Dec 23, 2025](https://finance.yahoo.com/news/stock-market-news-dec-23-103000908.html)  
- Analysts at Wedbush argue that investors are still underestimating how much AI and Azure can lift Microsoft’s revenue, and label the stock “undervalued” with a bullish 2026 outlook; this bullish AI narrative is a key driver supporting the stock on pullbacks. [Yahoo Finance – Is Microsoft Undervalued?](https://finance.yahoo.com/news/microsoft-undervalued-investors-tech-stock-203453041.html) [Yahoo Finance – Microsoft Set to Outpace AI Growth](https://ca.finance.yahoo.com/news/microsoft-set-outpace-ai-growth-180013478.html)  
- Recent commentary highlights strong AI‑driven cloud momentum, with estimates that expanding Azure and Copilot deployments could add roughly $25 billion to fiscal 2026 revenue versus current Street expectations, supporting higher long‑term price targets ($600+ in some cases). [Yahoo Finance – Microsoft Set to Outpace AI Growth](https://ca.finance.yahoo.com/news/microsoft-set-outpace-ai-growth-180013478.html) [Forbes – Microsoft Stock Eyes $600](https://www.forbes.com/sites/greatspeculations/2025/12/18/how-microsoft-stock-can-increase--to-600/)  
- On the fundamentals side, Microsoft’s most recent reported quarter (fiscal Q1 2026, which is the first quarter of its FY26) delivered 18% reported revenue growth to $77.7B and strong Azure growth (39%), underscoring that AI and cloud continue to be the main growth engines. [Yahoo Finance – Baron Q3 2025 letter excerpt on MSFT](https://finance.yahoo.com/news/microsoft-corporation-msft-azure-revenue-121646786.html)  
- The company is also announcing large incremental AI investment commitments globally (e.g., $17.5B for AI in India and $5.4B+ in Canada over the next few years), which investors view as both a cost headwind and a signal of long‑run AI revenue ambition. [Yahoo Finance – Heavy AI Investment Positions Microsoft](https://ca.finance.yahoo.com/news/heavy-ai-investment-positions-microsoft-172836323.html)  
- New AI partnerships and deployments (such as a multi‑year AI partnership with Cognizant and an Azure/OpenAI‑powered “Earth Copilot” with NASA) reinforce Microsoft’s positioning at the center of enterprise and public‑sector AI, supporting sentiment on the stock. [Yahoo Finance – Microsoft & Cognizant Partnership](https://finance.yahoo.com/news/microsoft-corporation-msft-cognizant-announce-145718967.html) [Nextgov – NASA & Microsoft Earth Copilot](https://www.nextgov.com/digital-government/2025/12/nasa-and-microsoft-finalize-tool-track-earths-water-changes/410238/)  

Overall, near‑term price action in MSFT looks driven by:  
- Sustained AI/cloud optimism and bullish analyst commentary  
- Strong recent earnings and AI‑related revenue momentum  
- Periodic volatility around AI adoption pace and macro/market moves  

---

2. **Key Facts with Citations**

**Recent price action / valuation**

- MSFT recently traded with a latest open of $484.88 and latest close of about $486.6, with a 2‑year total return of ~31.8% and 52‑week price change of ~+10.6%. [Yahoo Finance – Quote](https://finance.yahoo.com/quote/MSFT/) [Yahoo Finance – Key Statistics](https://finance.yahoo.com/quote/MSFT/key-statistics/)  
- Over the last two years the stock’s average close is about $438 and its standard deviation of close is about $45, implying moderate volatility for a mega‑cap. [Yahoo Finance – Quote](https://finance.yahoo.com/quote/MSFT/)  
- Recent trading days in late December 2025 show closes in a tight band around the high‑$480s (e.g., Dec 22 close $488.73; Dec 23 close $487.83), suggesting consolidation rather than a sharp move. [Yahoo Finance – Historical Data](https://finance.yahoo.com/quote/MSFT/history/)  
- A Zacks/Yahoo brief notes that on one recent day Microsoft closed at $483.84, up 1.62% versus the prior session, modestly outperforming the broader market. [Yahoo Finance – Microsoft Outpaces Stock Market Gains](https://finance.yahoo.com/news/microsoft-msft-outpaces-stock-market-224506000.html)  

**Market and analyst commentary driving the stock**

- A Yahoo Finance piece summarizing Wedbush’s view says analysts see “room to run” for MSFT in 2026 after a pullback from record highs, arguing that investors still “underestimate” Azure’s AI‑driven upside; Wedbush is bullish on the AI trade and sees continued records ahead. [Yahoo Finance – Is Microsoft Undervalued?](https://finance.yahoo.com/news/microsoft-undervalued-investors-tech-stock-203453041.html)  
- The same article notes that of 13 analysts tracked by Visible Alpha, 12 rate MSFT a “buy” and 1 a “hold,” with an average price target of $635, above Wedbush’s own bullish target, implying expectations of further upside. [Yahoo Finance – Is Microsoft Undervalued?](https://finance.yahoo.com/news/microsoft-undervalued-investors-tech-stock-203453041.html)  
- Another report cites Wedbush’s Dan Ives maintaining an “Outperform” rating with a $625 price target, estimating that AI‑driven Azure and Copilot deployments could lift Microsoft’s fiscal 2026 revenue by roughly $25 billion above the Street’s current view. [Yahoo Finance – Microsoft Set to Outpace AI Growth](https://ca.finance.yahoo.com/news/microsoft-set-outpace-ai-growth-180013478.html)  
- The Wedbush note emphasizes that Wall Street is “underestimating the Azure growth story and the AI‑driven shift,” framing Microsoft as a leader in enterprise AI with rising monetization. [Yahoo Finance – Microsoft Set to Outpace AI Growth](https://ca.finance.yahoo.com/news/microsoft-set-outpace-ai-growth-180013478.html)  
- A Forbes analysis argues Microsoft stock could reach $600 per share on continued growth in AI, cloud, and gaming, highlighting AI as a major earnings driver; this type of commentary contributes to bullish sentiment behind recent moves. [Forbes – Microsoft Stock Eyes $600](https://www.forbes.com/sites/greatspeculations/2025/12/18/how-microsoft-stock-can-increase--to-600/)  

**Earnings and guidance backdrop**

- An Investor‑letter summary (Baron Fifth Avenue Growth Fund) notes that in Microsoft’s first quarter of fiscal 2026, the company reported revenue of $77.7 billion, up 18% year‑over‑year (17% in constant currency), reflecting strong performance across lines including cloud. [Yahoo Finance – Baron letter excerpt on MSFT](https://finance.yahoo.com/news/microsoft-corporation-msft-azure-revenue-121646786.html)  
- The same source notes Azure revenue grew 39% in that period, underlining that AI and cloud workloads are a core driver of Microsoft’s current growth. [Yahoo Finance – Baron letter excerpt on MSFT](https://finance.yahoo.com/news/microsoft-corporation-msft-azure-revenue-121646786.html)  
- Consensus analyst estimates as of late December 2025 look for current‑quarter (Dec 2025) EPS of $3.86 (about +19.5% vs year‑ago EPS of $3.23) and revenue of about $80.3B, with full‑year FY26 revenue estimates around $326.8B and FY27 around $376B. [Yahoo Finance – MSFT Analysis/Earnings Estimates](https://finance.yahoo.com/quote/MSFT/analysis/) [Yahoo Finance – Investors Heavily Search MSFT](https://finance.yahoo.com/news/investors-heavily-search-microsoft-corporation-140004968.html)  
- An earnings‑call summary for an earlier fiscal 2025 quarter (Q2 FY25) shows Microsoft delivering revenue of $69.6B, up 12% year‑over‑year, with EPS of $3.23 (up 10%) and Microsoft Cloud revenue of $40.9B (+21%), plus an AI run‑rate metric the company began highlighting; this contextualizes the acceleration into FY26. [Yahoo Finance – MSFD.XD Q2 FY2025 Earnings Call Summary](https://finance.yahoo.com/quote/MSFD.XD/earnings/MSFD.XD-Q2-2025-earnings_call-251674.html/)  

**Recent news / announcements (especially AI‑related)**

- A Yahoo Finance article notes Microsoft is making a “massive push” into AI and, in December 2025, announced plans to spend $17.5 billion on AI development in India and over $5.4 billion in Canada over the next few years, part of a broader strategy that could increase sales nearly 80% over five years relative to its more than $280 billion in fiscal 2025 revenue. [Yahoo Finance – Heavy AI Investment Positions Microsoft](https://ca.finance.yahoo.com/news/heavy-ai-investment-positions-microsoft-172836323.html)  
- The same piece argues that these AI investments could “meaningfully reshape the long‑term outlook for its stock,” reinforcing the narrative that AI remains a major growth engine for Microsoft beyond 2026. [Yahoo Finance – Heavy AI Investment Positions Microsoft](https://ca.finance.yahoo.com/news/heavy-ai-investment-positions-microsoft-172836323.html)  
- On December 18, 2025, Microsoft and Cognizant announced a multi‑year partnership to help enterprises in healthcare, life sciences, retail, financial services and manufacturing “harness” AI‑powered solutions, expanding Cognizant’s Neuro AI suite built on Microsoft’s cloud and AI services. [Yahoo Finance – Microsoft & Cognizant Partnership](https://finance.yahoo.com/news/microsoft-corporation-msft-cognizant-announce-145718967.html)  
- In that announcement, Microsoft commercial business CEO Judson Althoff said that combining Microsoft’s cloud and “agentic AI capabilities” with Cognizant’s platforms and delivery scale would accelerate creation of industry‑specific solutions and unlock “transformative value” for joint customers. [Yahoo Finance – Microsoft & Cognizant Partnership](https://finance.yahoo.com/news/microsoft-corporation-msft-cognizant-announce-145718967.html)  
- Microsoft is also rolling out AI deployments in the public sector: NASA and Microsoft finalized an “Earth Copilot” tool to help track global water changes, implemented as a multi‑agent AI system on Microsoft’s Azure OpenAI platform and Foundry framework, to make hydrology data more accessible via natural‑language queries. [Nextgov – NASA & Microsoft Earth Copilot](https://www.nextgov.com/digital-government/2025/12/nasa-and-microsoft-finalize-tool-track-earths-water-changes/410238/)  

**AI products and ecosystem positioning**

- Articles about Copilot+ PCs note that Microsoft introduced these AI‑centric PCs in mid‑2024 with dedicated NPUs capable of ~40 trillion operations per second to support on‑device AI workloads, together with a Windows AI Foundry framework for local and hybrid AI applications. [EdTech Magazine – Copilot+ PCs](https://edtechmagazine.com/higher/article/2025/12/copilot-pcs-offer-fast-powerful-ai-boost-faculty-members-productivity)  
- This hardware and software stack (Copilot+ PCs, NPUs, Windows AI Foundry) is being promoted as a way to enhance productivity, including in higher education, indirectly reinforcing Microsoft’s role as a platform provider for AI at the PC edge in addition to the cloud. [EdTech Magazine – Copilot+ PCs](https://edtechmagazine.com/higher/article/2025/12/copilot-pcs-offer-fast-powerful-ai-boost-faculty-members-productivity)  
- Several recent AI‑themed commentaries cluster Microsoft among the top “AI leaders” and “top tech dividend stocks,” linking its dividend plus AI growth profile to institutional popularity; an article notes that 312 hedge fund portfolios held MSFT at the end of Q3 2025, up from 294 in the prior quarter. [Yahoo Finance – Baron letter excerpt on MSFT](https://finance.yahoo.com/news/microsoft-corporation-msft-azure-revenue-121646786.html) [Yahoo Finance – Heavy AI Investment Positions Microsoft](https://ca.finance.yahoo.com/news/heavy-ai-investment-positions-microsoft-172836323.html)  

**Short‑term headwinds / volatility sources**

- A Yahoo Finance note indicates that on one recent day Microsoft stock fell more than 2% to about $479 following reports that it had reduced AI software sales efforts, amid concerns about slower‑than‑hoped corporate adoption of AI; such headlines can spark short‑term pullbacks even amid a strong longer‑term AI story. [Yahoo Finance – Why Microsoft Stock Is Falling](https://finance.yahoo.com/news/why-microsoft-msft-stock-falling-192859238.html)  
- Broader market coverage indicates that U.S. equity markets were generally positive heading into Christmas week 2025, with seasonal strength and optimism helping large‑cap tech stocks like Microsoft, contributing to its recent modest gains. [Yahoo Finance – Stock Market News Dec 23, 2025](https://finance.yahoo.com/news/stock-market-news-dec-23-103000908.html)  

**Bull vs. bear factors embedded in the current move (interpretation, grounded in sources)**

*Bullish factors (supporting the stock)*  
- Strong and accelerating cloud/AI fundamentals: 39% Azure growth in the latest reported quarter and 18% overall revenue growth underscore robust demand for AI‑related services. [Yahoo Finance – Baron letter excerpt on MSFT](https://finance.yahoo.com/news/microsoft-corporation-msft-azure-revenue-121646786.html)  
- AI‑driven upside to consensus: Wedbush and others see potential for ~$25B extra fiscal 2026 revenue from AI versus Street expectations, and price targets in the $600–$635 range, above current levels. [Yahoo Finance – Microsoft Set to Outpace AI Growth](https://ca.finance.yahoo.com/news/microsoft-set-outpace-ai-growth-180013478.html) [Yahoo Finance – Is Microsoft Undervalued?](https://finance.yahoo.com/news/microsoft-undervalued-investors-tech-stock-203453041.html) [Forbes – Microsoft Stock Eyes $600](https://www.forbes.com/sites/greatspeculations/2025/12/18/how-microsoft-stock-can-increase--to-600/)  
- Strategic AI investments and partnerships (India, Canada, Cognizant, NASA) position Microsoft deeply into both private and public‑sector AI workloads, cementing ecosystem dominance. [Yahoo Finance – Heavy AI Investment Positions Microsoft](https://ca.finance.yahoo.com/news/heavy-ai-investment-positions-microsoft-172836323.html) [Yahoo Finance – Microsoft & Cognizant Partnership](https://finance.yahoo.com/news/microsoft-corporation-msft-cognizant-announce-145718967.html) [Nextgov – NASA & Microsoft Earth Copilot](https://www.nextgov.com/digital-government/2025/12/nasa-and-microsoft-finalize-tool-track-earths-water-changes/410238/)  

*Bearish / risk factors*  
- Valuation risk: With a 2‑year total return of ~32% and ongoing optimism priced in, the stock trades at a premium, and any disappointment in AI monetization or Azure growth could trigger corrections, as illustrated by the >2% drop on concerns about slower corporate AI adoption. [Yahoo Finance – Quote](https://finance.yahoo.com/quote/MSFT/) [Yahoo Finance – Why Microsoft Stock Is Falling](https://finance.yahoo.com/news/why-microsoft-msft-stock-falling-192859238.html)  
- Execution and adoption risk: Reports of reduced AI sales efforts and slower adoption in some corporate segments show that the path from AI hype to broad revenue realization can be uneven, creating intermittent volatility. [Yahoo Finance – Why Microsoft Stock Is Falling](https://finance.yahoo.com/news/why-microsoft-msft-stock-falling-192859238.html)  

**Upcoming / ongoing catalysts**

- Next earnings: Analysts expect the current quarter (Dec 2025) to show strong double‑digit EPS and revenue growth; the actual earnings release and AI commentary will likely be key near‑term drivers of the stock. [Yahoo Finance – MSFT Analysis/Earnings Estimates](https://finance.yahoo.com/quote/MSFT/analysis/)  
- AI monetization updates: Any new disclosure on Copilot adoption, AI run‑rate revenue, and enterprise AI deals (similar to the Cognizant and NASA partnerships) are likely to influence sentiment around MSFT’s AI premium. [Yahoo Finance – Microsoft & Cognizant Partnership](https://finance.yahoo.com/news/microsoft-corporation-msft-cognizant-announce-145718967.html) [Nextgov – NASA & Microsoft Earth Copilot](https://www.nextgov.com/digital-government/2025/12/nasa-and-microsoft-finalize-tool-track-earths-water-changes/410238/)  
- Capital allocation and investment signals: Further large AI infrastructure investment announcements, region‑specific AI spending plans, or commentary on returns from the $17.5B India and $5.4B Canada AI initiatives could shape long‑term growth expectations. [Yahoo Finance – Heavy AI Investment Positions Microsoft](https://ca.finance.yahoo.com/news/heavy-ai-investment-positions-microsoft-172836323.html)  

---

3. **Citations List**

1. [Yahoo Finance – MSFT Quote](https://finance.yahoo.com/quote/MSFT/) – Recent price, 2‑year performance metrics, volatility.  
2. [Yahoo Finance – MSFT Historical Data](https://finance.yahoo.com/quote/MSFT/history/) – Daily price history in December 2025.  
3. [Yahoo Finance – MSFT Key Statistics](https://finance.yahoo.com/quote/MSFT/key-statistics/) – 52‑week change and other valuation measures.  
4. [Yahoo Finance – Stock Market News for Dec 23, 2025](https://finance.yahoo.com/news/stock-market-news-dec-23-103000908.html) – Broader market context and year‑end rally.  
5. [Yahoo Finance – Microsoft Outpaces Stock Market Gains](https://finance.yahoo.com/news/microsoft-msft-outpaces-stock-market-224506000.html) – Note on a recent day’s relative outperformance.  
6. [Yahoo Finance – Is Microsoft Undervalued by Investors?](https://finance.yahoo.com/news/microsoft-undervalued-investors-tech-stock-203453041.html) – Wedbush view, 2026 outlook, analyst ratings and targets.  
7. [Yahoo Finance – Microsoft Set to Outpace AI Growth](https://ca.finance.yahoo.com/news/microsoft-set-outpace-ai-growth-180013478.html) – Wedbush AI thesis, $25B potential revenue uplift, $625 PT.  
8. [Forbes – Microsoft Stock Eyes $600](https://www.forbes.com/sites/greatspeculations/2025/12/18/how-microsoft-stock-can-increase--to-600/) – Analysis of path to $600 driven by AI, cloud, gaming.  
9. [Yahoo Finance – Baron Fifth Avenue Growth Fund Letter Excerpt on MSFT](https://finance.yahoo.com/news/microsoft-corporation-msft-azure-revenue-121646786.html) – Q1 FY26 revenue growth, Azure 39% growth, hedge‑fund ownership count.  
10. [Yahoo Finance – MSFT Analysis / Earnings Estimates](https://finance.yahoo.com/quote/MSFT/analysis/) – Consensus EPS and revenue estimates for quarters and FY26–27.  
11. [Yahoo Finance – Investors Heavily Search Microsoft Corporation (MSFT)](https://finance.yahoo.com/news/investors-heavily-search-microsoft-corporation-140004968.html) – Expected EPS growth vs prior year.  
12. [Yahoo Finance – MSFD.XD Q2 FY2025 Earnings Call Summary](https://finance.yahoo.com/quote/MSFD.XD/earnings/MSFD.XD-Q2-2025-earnings_call-251674.html/) – Earlier FY25 revenue, EPS, cloud, and AI run‑rate commentary (context).  
13. [Yahoo Finance – Heavy AI Investment Positions Microsoft (MSFT)](https://ca.finance.yahoo.com/news/heavy-ai-investment-positions-microsoft-172836323.html) – AI investment plans in India and Canada, long‑term AI growth framing.  
14. [Yahoo Finance – Microsoft & Cognizant Announce Multi‑Year Partnership](https://finance.yahoo.com/news/microsoft-corporation-msft-cognizant-announce-145718967.html) – AI partnership details, Althoff quote.  
15. [Nextgov/FCW – NASA and Microsoft Finalize Earth Copilot](https://www.nextgov.com/digital-government/2025/12/nasa-and-microsoft-finalize-tool-track-earths-water-changes/410238/) – Azure/OpenAI‑powered Earth Copilot AI tool.  
16. [EdTech Magazine – Copilot+ PCs Offer Fast, Powerful AI](https://edtechmagazine.com/higher/article/2025/12/copilot-pcs-offer-fast-powerful-ai-boost-faculty-members-productivity) – Copilot+ PCs, NPU specs, Windows AI Foundry framework.  
17. [Yahoo Finance – Why Microsoft Stock Is Falling](https://finance.yahoo.com/news/why-microsoft-msft-stock-falling-192859238.html) – 2%+ drop on slower corporate AI adoption / reduced AI sales efforts.  

---

4. **Gaps in the research**

- I did not find a single, clearly dominant one‑day catalyst (like a major legal ruling or CEO change) for MSFT’s most recent small price moves; action seems driven by a mix of overall market trends and ongoing AI narratives.  
- Exact intra‑day price/volume and more granular order‑flow data for very recent sessions would require access to a real‑time market data terminal (e.g., Bloomberg, Refinitiv), which I did not access here.  
- Detailed internal metrics on Copilot user counts, AI revenue run‑rate and margins are only partly disclosed in earnings materials and are not fully available in the public summaries I cited.

---
# Example 2: Sector Analysis (Medium)

**Context**: A portfolio manager wants to understand sector-wide trends affecting their holdings.

**Sub-agent role**: Research the semiconductor sector, identify key trends, and gather relevant data.

In [ ]:
# Example 2: Sector analysis
example_2_message = """Research the current state of the semiconductor sector.

I need to understand:
1. How major semiconductor stocks have performed recently (NVDA, AMD, INTC, AVGO)
2. Key trends driving the sector (AI demand, supply chain, geopolitics)
3. Recent analyst sentiment and price target changes
4. Any upcoming catalysts or risks

Save any relevant data you find to the scratchpad/data/ folder.
Provide properly cited sources for all claims.
"""

response_2 = test_web_research_agent(example_2_message)


---
# Example 3: Comprehensive Event Research (Complex)

**Context**: A risk manager needs to understand the market implications of a major economic event.

**Sub-agent role**: Research across multiple asset classes and provide a comprehensive view with data.

In [ ]:
# Example 3: Comprehensive event research
example_3_message = """Research the market impact of the most recent Federal Reserve interest rate decision.

I need comprehensive research covering:

1. **The Decision Itself**
   - What was the decision (rate change, guidance)
   - How did it compare to market expectations
   - Key quotes from Fed Chair's statement

2. **Market Reactions Across Asset Classes**
   - Equity indices (S&P 500, Nasdaq, small caps)
   - Bond yields (2Y, 10Y treasuries)
   - Dollar index and gold

3. **Sector-Specific Impacts**
   - Which sectors benefited vs suffered
   - Rate-sensitive stocks (banks, REITs, utilities)
   - Growth vs value performance

4. **Forward-Looking Implications**
   - Market pricing for future rate decisions
   - Analyst commentary on the path forward
   - Risks and uncertainties

Save any relevant data or notes to the scratchpad folders.
Every claim must have a source citation.
"""

response_3 = test_web_research_agent(example_3_message)


---
# Verify Output Files

Check what data and notes were saved to the scratchpad directories.

In [ ]:
import os
from pathlib import Path

scratchpad_dir = Path("../scratchpad")

# Check data directory
data_dir = scratchpad_dir / "data"
if data_dir.exists():
    data_files = list(data_dir.glob("*"))
    if data_files:
        print(f"Found {len(data_files)} files in scratchpad/data/:")
        for f in sorted(data_files, key=lambda x: x.stat().st_mtime, reverse=True):
            print(f"  - {f.name}")
    else:
        print("scratchpad/data/ is empty")

print()

# Check notes directory
notes_dir = scratchpad_dir / "notes"
if notes_dir.exists():
    notes_files = list(notes_dir.glob("*"))
    if notes_files:
        print(f"Found {len(notes_files)} files in scratchpad/notes/:")
        for f in sorted(notes_files, key=lambda x: x.stat().st_mtime, reverse=True):
            print(f"  - {f.name}")
    else:
        print("scratchpad/notes/ is empty")

---
# Notes

## Expected Outputs

For each example, the web research agent should provide:

### 1. Summary of Findings
- Each statement with inline citation: [Source Name](URL)

### 2. Key Facts with Citations
- Format: "Fact statement [Source Name](URL)"
- No uncited claims

### 3. Citations List
- Numbered list of all sources used
- Brief description of what info came from each source

### 4. Gaps in Research
- What couldn't be found or verified

## Data Output

When the agent finds useful data, it should save to:
- `scratchpad/data/` - CSV files, datasets for analysis
- `scratchpad/notes/` - Detailed research notes

## Best Practices Being Tested

- Multiple search queries to triangulate information
- Noting the date of sources (recency matters)
- Distinguishing between news, opinion, and research
- Flagging unreliable sources

## Integration with Main Agent

In production, the main agent would:
1. Assign research tasks to `web-research-agent`
2. Receive findings with citations
3. Send findings to `credibility-agent` for verification
4. Pass verified data to `analysis-agent` for processing
5. Compile everything into final reports